In [258]:
import pandas as pd
import numpy as np
import os
import random

In [204]:
os.chdir("/Users/jothiramaswamy/Downloads/irish")

In [328]:
with open('ED-00001-00000001-soi.txt') as f:
    lines = [line.strip().split(",") for line in f.readlines()]

In [329]:
candidate_count = int(lines.pop(0)[0])

In [330]:
candidate_count

12

In [331]:
candidates = {}
for i in range(candidate_count):
    candidate = lines.pop(0)
    candidates[int(candidate[0])] = candidate[1]

In [332]:
candidates

{1: 'Cathal Boland F.G.',
 2: 'Clare Daly S.P.',
 3: 'Mick Davis S.F.',
 4: 'Jim Glennon F.F.',
 5: 'Ciaran Goulding Non-P',
 6: 'Michael Kennedy F.F.',
 7: 'Nora Owen F.G.',
 8: 'Eamonn Quinn Non-P',
 9: 'Sean Ryan Lab',
 10: 'Trevor Sargent G.P.',
 11: 'David Henry Walshe C.C. Csp',
 12: 'G.V. Wright F.F.'}

In [333]:
vote_count = int(lines.pop(0)[0])
vote_count

43942

In [335]:
preferences = []
for line in lines:
    indif = []
    indifferent = False
    pref = []
    for l in line:
        if l[0] == '{':
            indif.append(int(l[1:]))
            indifferent = True
        elif l[-1] == '}':
            indif.append(int(l[:-1]))
            indifferent = False
            random.shuffle(indif)
            pref += indif
            indif = []
        elif indifferent:
            indif.append(int(l))
        else:
            pref.append(int(l))
    preferences.append(pref)

In [336]:
class RankedChoice:

    def __init__(self, candidates, vote_count, preferences, pref_type):
        self.candidates = candidates
        self.vote_count = vote_count
        self.preferences = preferences
        self.pref_type = pref_type

    def determine_winner(self):
        vote_tally = [0] * len(self.candidates)
        candidates_left = list(self.candidates.keys())
        preferences = self.preferences.copy()
        while True:
            for p in range(len(preferences)):
                if len(preferences[p]) == 1:
                    continue
                try:
                    while preferences[p][1] not in candidates_left:
                        preferences[p].pop(1)
                        if len(preferences[p]) == 1:
                            continue
                except:
                    continue
                idx = candidates_left.index(preferences[p][1])
                vote_tally[idx]+=preferences[p][0]
                
            if max(vote_tally) == min(vote_tally):
                return "Tie"
            elif max(vote_tally) > sum(vote_tally)/2:
                idx = vote_tally.index(max(vote_tally))
                winner_idx = candidates_left[idx]
                return self.candidates[winner_idx]
            else:
                idx = vote_tally.index(min(vote_tally))
                candidates_left.pop(idx)
                vote_tally = [0] * len(candidates_left)


In [337]:
class Plurality:

    def __init__(self, candidates, vote_count, preferences, pref_type):
        self.candidates = candidates
        self.vote_count = vote_count
        self.preferences = preferences
        self.pref_type = pref_type

    def determine_winner(self):
        vote_tally = [0] * len(self.candidates)
        preferences = self.preferences.copy()
        for p in range(len(preferences)):
            try:
                vote_tally[preferences[p][1]] += preferences[p][0]
            except:
                continue

        idx = vote_tally.index(max(vote_tally))
        return self.candidates[idx]

In [338]:
class Borda:

    def __init__(self, candidates, vote_count, preferences, pref_type):
        self.candidates = candidates
        self.vote_count = vote_count
        self.preferences = preferences
        self.pref_type = pref_type

    def determine_winner(self):
        vote_tally = [0] * len(self.candidates)
        preferences = self.preferences.copy()
        for p in range(len(preferences)):
            for i in range(1, len(preferences[p])):
                try:
                    vote_tally[preferences[p][i]-1] += float(preferences[p][0])*(1/i)
                except:
                    print(preferences[p][i], len(vote_tally))

        idx = vote_tally.index(max(vote_tally))
        return self.candidates[idx+1]

In [339]:
choice = RankedChoice(candidates, vote_count, preferences, "soi")

In [340]:
choice.determine_winner()

'Trevor Sargent G.P.'

In [341]:
choice = Plurality(candidates, vote_count, preferences, "soi")

In [342]:
choice.determine_winner()

'Trevor Sargent G.P.'

In [343]:
choice = Borda(candidates, vote_count, preferences, "soi")

In [344]:
choice.determine_winner()

'Trevor Sargent G.P.'